Import Libraries

In [1]:
import tensorflow as tf
import tensorflow.keras
import matplotlib.pylab as plt
import numpy as np
import os
import shutil
import albumentations as A
import cv2
from sklearn.model_selection import train_test_split

SEED = 42
import random 
random.seed(SEED)
import numpy as np
np.random.seed(SEED)
import tensorflow as tf
tf.random.set_seed(SEED)

# ** Inputs ** 

In [2]:
# New Augmented Folder name
new_folder_name = 'FMD_aug_tvt'

# Name of folder where the regular folders are
source_folder = 'FMD_tvt'

# Mulitplier for the train images
train_multiplier = 10

# Mulitplier for the train images
val_multiplier = 5

In [3]:
tvt_folders = ['train', 'val', 'test']
im_folders = ['image', 'mask']

Fmd_class_path = os.path.join(source_folder, tvt_folders[0], im_folders[0])
classes = os.listdir(os.path.join(Fmd_class_path))
print(classes)

['fabric', 'foliage', 'glass', 'leather', 'metal', 'paper', 'plastic', 'stone', 'water', 'wood']


# Augment Function

In [4]:
transform = A.Compose([
    A.RandomCrop(width=256, height=256),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RGBShift(p=0.4),
    A.ColorJitter(p=0.2),
    A.RandomBrightnessContrast(p=0.2)
])

# Making the new folder

In [5]:
def make_FMD_tvt(file_name, classes, tvt_folders, im_folders):
    if not os.path.exists(os.path.join(file_name)):
        for tvt in tvt_folders:
            for im in im_folders:
                for class_ in classes:
                    os.makedirs(os.path.join(file_name, tvt, im, class_))

Running the function

In [6]:
make_FMD_tvt(new_folder_name, classes, tvt_folders, im_folders)

# Splitting Train and Test from FMD to New folder

In [7]:
def move_images_to_new(class_name, new_folder_name, tvt_folders, im_folders):
    train_imgs = os.listdir(os.path.join(source_folder, tvt_folders[0], im_folders[0], class_name))
    val_imgs = os.listdir(os.path.join(source_folder, tvt_folders[1], im_folders[0], class_name))
    test_imgs = os.listdir(os.path.join(source_folder, tvt_folders[2], im_folders[0], class_name))

    tvt_list = [train_imgs, val_imgs, test_imgs]

    for i in range(len(tvt_folders)):
        for j in range(len(tvt_list[i])):
            source_path = os.path.join(source_folder, tvt_folders[i], im_folders[0], class_name, tvt_list[i][j])
            destination_path = os.path.join(new_folder_name, tvt_folders[i], im_folders[0], class_name)
            shutil.copy(source_path, destination_path)
    return
  

Running function

In [8]:
#Split train stuff accordingly
for i in range(len(classes)):
  move_images_to_new(classes[i], new_folder_name, tvt_folders, im_folders)

# Moving the masks to New Folder

In [9]:
def move_masks(class_name, new_folder_name, tvt_folders, im_folders):
    train_list = os.listdir(os.path.join(new_folder_name, tvt_folders[0], im_folders[0], class_name))
    val_list = os.listdir(os.path.join(new_folder_name, tvt_folders[1], im_folders[0], class_name))
    test_list = os.listdir(os.path.join(new_folder_name, tvt_folders[2], im_folders[0], class_name))

    tvt_list = [train_list, val_list, test_list]
  
    for i in range(len(tvt_list)):
        for j in range(len(tvt_list[i])):
            source_path = os.path.join(source_folder, tvt_folders[i], im_folders[1], class_name, tvt_list[i][j])
            destination_path = os.path.join(new_folder_name, tvt_folders[i], im_folders[1], class_name)
            if os.path.exists(source_path):
                shutil.copy(source_path, destination_path)
    return

Running Function

In [10]:
#Split train stuff accordingly
for i in range(len(classes)):
  move_masks(classes[i], new_folder_name, tvt_folders, im_folders)

# Data Augmentation Function

In [11]:
def data_augmentation(class_name, new_folder_name, tvt_folders, im_folders, multiplier, tvt_num):
    print(f'Starting {class_name}')
    images_in_folder = os.listdir(os.path.join(new_folder_name, tvt_folders[tvt_num], im_folders[0], class_name))
    for image_name in images_in_folder:
        img = cv2.imread(os.path.join(new_folder_name, tvt_folders[tvt_num], im_folders[0], class_name, image_name))
        msk = cv2.imread(os.path.join(new_folder_name, tvt_folders[tvt_num], im_folders[1], class_name, image_name))

        store_path_image = os.path.join(new_folder_name, tvt_folders[tvt_num], im_folders[0], class_name)
        store_path_mask = os.path.join(new_folder_name, tvt_folders[tvt_num], im_folders[1], class_name)

        for i in range(multiplier):
            transformed = transform(image = img, mask = msk)
      
            transformed_image = transformed['image']
            transformed_mask = transformed['mask']

            cv2.imwrite(os.path.join(store_path_image, f'{image_name[0:-4]}_{i}.jpg'), transformed_image)
            cv2.imwrite(os.path.join(store_path_mask, f'{image_name[0:-4]}_{i}.jpg'), transformed_mask)
    
    print(f"Success {class_name}")
        

Running Function

In [12]:
for i in range(len(classes)):
  data_augmentation(classes[i], new_folder_name, tvt_folders, im_folders, train_multiplier, 0)
  data_augmentation(classes[i], new_folder_name, tvt_folders, im_folders, val_multiplier, 1)

Starting fabric
Success fabric
Starting fabric
Success fabric
Starting foliage
Success foliage
Starting foliage
Success foliage
Starting glass
Success glass
Starting glass
Success glass
Starting leather
Success leather
Starting leather
Success leather
Starting metal
Success metal
Starting metal
Success metal
Starting paper
Success paper
Starting paper
Success paper
Starting plastic
Success plastic
Starting plastic
Success plastic
Starting stone
Success stone
Starting stone
Success stone
Starting water
Success water
Starting water
Success water
Starting wood
Success wood
Starting wood
Success wood
